In [1]:
import numpy as np
#import delong
import pandas as pd
import re
import matplotlib as mpl
import matplotlib.pyplot as plt
#import fasttext

#from gensim.models.wrappers import FastText
#from gensim.models import KeyedVectors

from sklearn.model_selection import cross_validate, StratifiedKFold, RepeatedStratifiedKFold
from sklearn import svm, linear_model
from sklearn import dummy
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import scale
import scipy
import scipy.stats
from sklearn.metrics import make_scorer, roc_curve, precision_recall_curve, average_precision_score, accuracy_score, roc_auc_score


In [2]:
#Adapted from https://bitbucket.org/bopjesvla/thesis/src/master/src/combined.py

#wv = [("FastText", "wiki"), ("GloVe", "wiki")]

cnt = lambda l, words: sum(l.count(w) for w in words)
cnt_phrases = lambda l, phrases: sum(l[i:i+len(p)]==p for p in phrases for i in range(len(l)))
contains_any = lambda l, words: sum(any(punc.find(w) != -1 for w in words) for punc in l)
not_contractions = [c.split("'") for c in "can't|couldn't|didn't|doesn't|don't|hadn't|hasn't|haven't|isn't|weren't|wasn't|shouldn't|aren't|won't".split("|")]
other_negations = "cannot|neither|never|no|nobody|none|nope|nor|not|nothing|nowhere|uhuh|without|wouldn|zero|zip".split("|")
neg_emotion = re.compile(r"^(abandon.*|abuse.*|abusive|ache.*|aching|advers.*|afraid|aggravat.*|aggress.*|agitat.*|agony|alarm.*|alone|anger.*|angr.*|anguish.*|annoy.*|antagoni.*|anxi.*|appall.*|apprehens.*|argu.*|arrogan.*|asham.*|assault.*|aversi.*|avoid.*|awful|bad|bastard|beaten|bewilder.*|bitch.*|bitter.*|blam.*|bore.*|boring|bother.*|burden.*|careless.*|cheat.*|complain.*|confus.*|contradic.*|crap.*|craz.*|cried|cries|critical|critici.*|cruel.*|crushed|cry|crying|cut|cynical|damn.*|danger.*|daze.*|decay.*|defeat.*|defect.*|defens.*|degrad.*|depress.*|depriv.*|despair.*|desperate.*|despis.*|destroy.*|destruct.*|devastat.*|devil.*|difficult.*|disagree.*|disappoint.*|disaster.*|discomfort.*|discourag.*|disgust.*|dislike|disliked|dislikes|dismay.*|distraught|distress.*|distrust.*|disturb.*|dominate.*|doom.*|doubt.*|dread.*|dull.*|dumb.*|dump.*|dwell.*|egotis.*|embarass.*|emotional|empt.*|enem.*|enrag.*|envious|envy|evil|excruciat.*|exhaust.*|fail.*|fatal|fatigu.*|fear|feared|fearing|fears|feud.*|fight|fighting|fights|flop.*|flunk.*|forbid.*|fought|frantic.*|freak.*|fright.*|frustrat.*|fuck.*|furious.*|gloom.*|goddam.*|gossip.*|grave.*|greed.*|grief|griev.*|grim.*|grind|gross.*|guilt.*|harass.*|hate|hated|hateful|hates|hating|hatred|hazy|hell|helpless.*|hesitant|homesick.*|hopeless.*|horribl.*|horrif.*|horror|hostil.*|humiliat.*|hurt.*|ignoran.*|impatien.*|impersonal|inadequate|indifferen.*|ineffect.*|inferior|inhib.*|insecur.*|insult.*|interrup.*|intimidat.*|irrational|irrita.*|isolat.*|jealous.*|jerk|jerked|jerks|kill.*|lame|liar.*|lie|lied|lies|loneli.*|lonely|lonesome|longing|lose|loser.*|losing|loss.*|lost|lous.*|low.*|ludicrous.*|mad|mess|messy|miser.*|miss|missed|misses|missing|molest.*|moody|mourn.*|nag.*|nast.*|neglect.*|nervous.*|numb|obnoxious.*|obsess.*|offend.*|outrag.*|overwhelm.*|pain|painf.*|painl.*|pains|panic.*|paranoi.*|pathetic.*|peculiar.*|pervert.*|pessimis.*|petrif.*|pett.*|piss.*|pitiful.*|pity|poison.*|prejudic.*|pressur.*|protest|protested|protesting|puk.*|punish.*|rage.*|rape.*|rebel.*|regret.*|reject.*|reluctan.*|remorse.*|repress.*|resent.*|resign.*|restless.*|revenge.*|ridicul.*|rigid.*|rude.*|ruin.*|sad|sarcas.*|scare.*|scream.*|screw.*|selfish.*|serious.*|severe.*|shak.*|shame.*|shit.*|shock.*|shy.*|sicken.*|silly|sin|sinister|sins|skeptical|smother.*|snob.*|sorrow.*|sorry|spite.*|startl.*|strain.*|strange|stress.*|stubborn.*|stunned|stuns|stupid|suck|sucked|sucking|sucks|suffer|suffered|suffering|suffers|suspicious.*|tear.*|teas.*|temper|tense.*|tension.*|terribl.*|terrified.*|terrifying|terror.*|threaten.*|tick|ticked|torture.*|tragedy|tragic|trembl.*|trick.*|troubl.*|turmoil|ugh|ugly|unattractive|uncertain|uncomfortable|uneas.*|unfortunate.*|unhapp.*|unimportant|unpleasant|unprotected|unsuccessful|unsure.*|upset.*|useless|vain|vanity|vicious.*|victim.*|violent.*|vulnerab.*|weak.*|weep.*|weird.*|whine.*|wicked.*|worr.*|worse.*|worthless|wrong.*)$")
sensory = re.compile(r"^(appear|appeared|appearing|appears|ask|asked|asking|asks|ate|bitter*|call|called|calling|calls|chat.*|contact.*|discuss.*|drank|drink|drinking|drinks|ear|ears|eat|eaten|eating|eats|eye.*|feel|feeling.*|feels|felt|grab.*|handl.*|hear|heard|hearing|hears|held|hold|holding|holds|hug.*|itch.*|listen|listened|listening|listens|look|looked|looking|looks|noise.*|observ.*|pain|painf.*|painl.*|pains|perceiv.*|perception.*|read|reading|reads|rub|rubbed|rubs|said|saw|say.*|see|seeing|seen|sees|sensation|sensations|sense|sensed|senses|sensing|show|showed|showing|shows|sight.*|skin|smell.*|sound.*|speak|speaking|speaks|spoke.*|squeez.*|stare.*|sweet|talk|talked|talking|talks|tast.*|tell|telling|tells|told|touch.*|view|viewed|viewing|views|vision.*|visual|watch*|witness)$")
anger = re.compile(r"^(abuse.*|abusive|aggravat.*|aggress.*|agitat.*|anger.*|angr.*|annoy.*|antagoni.*|argu.*|arrogan.*|assault.*|bastard|beaten|bitch.*|bitter.*|blam.*|cheat.*|contradic.*|crap.*|critical|critici.*|cruel.*|cut|cynical|damn.*|danger.*|defens.*|despis.*|destroy.*|destruct.*|disgust.*|distrust.*|dominate.*|dread.*|dumb.*|dump.*|enem.*|enrag.*|evil|feud.*|fight|fighting|fights|fought|frustrat.*|fuck.*|furious.*|goddam.*|greed.*|harass.*|hate|hated|hateful|hates|hating|hatred|hostil.*|humiliat.*|insult.*|interrup.*|intimidat.*|jealous.*|jerk|jerked|jerks|kill.*|liar.*|lied|lous.*|ludicrous.*|mad|molest.*|nag.*|nast.*|obnoxious.*|offend.*|outrag.*|paranoi.*|piss.*|poison.*|prejudic.*|punish.*|rage.*|rape.*|rebel.*|resent.*|revenge.*|ridicul.*|rude.*|sarcas.*|screw.*|shit.*|sinister|skeptical|smother.*|snob.*|spite.*|stubborn.*|stupid|suck|sucked|sucking|sucks|suspicious.*|teas.*|temper|terrified.*|terrifying|terror.*|threaten.*|tick|ticked|torture.*|trick.*|ugly|vicious.*|victim.*|violent.*|wicked)$")

spp = "thee thine thou thoust thy yall ya ye you your".split()
quantifiers = 'all some much many little few any lots lot plenty several more most less least none'.split()
insight = re.compile(r"^(accept|acknowledg.*|adjust.*|admit|admits|admitted|admitting|analys.*|analyz.*|answer.*|aware.*|became|believe|believed|believes|believing|clarif.*|clear|closure|cohere.*|compreh.*|concentrat.*|conclud.*|conclus.*|confess.*|construct.*|create.*|creating|decid.*|determina.*|determine|determined|determines|determining|discern.*|discl.*|discover.*|effect.*|enlighten.*|evaluat.*|examine.*|examining|explain|explained|explaining|explains|explanat.*|explor.*|feeling.*|feels|felt|figur.*|find.*|forgiv.*|found|gather.*|generate.*|grasp.*|inform|informs|insight.*|knew|know|knowing|knowl.*|known|knows|learn.*|meaning|meaningf.*|means|meant|motivate.*|perceiv.*|perception.*|ponder.*|question|questioning|questionned|questions|rational.*|realiz.*|reason.*|reckon.*|recognis.*|recogniz.*|reconsider.*|reconstruct.*|reflect.*|relate.*|relation.*|resolu.*|resolve|resolved|rethink.*|reveal.*|saw|secret|secrets|see|seeing|solution.*|solve.*|suspect.*|think|thinking|thinks|thought|thoughts|understand|understandable|understanding|understands|understood|wonder|wondered|wondering)$")
cognitive = re.compile(r"abandon.*|accept|accepted|accepting|accepts|achiev.*|acknowledg.*|adjust.*|admit|admits|admitted|admitting|affect|affected|affects|agree.*|anal|analys.*|analyz.*|answer.*|approv.*|arrange.*|assum.*|avoid.*|aware.*|barrier.*|bases|basis|became|because|become|becomes|becoming|believe|believed|believes|believing|block.*|brake.*|but|careful.*|caus.*|clarif.*|clear|clog.*|closure|cohere.*|complete|compreh.*|concentrat.*|concern.*|conclud.*|conclus.*|confess.*|confide|confided|confides|confiding|confirm.*|conflict.*|confus.*|consequen.*|constrain.*|constrict.*|construct.*|contain.*|contradic.*|control.*|cos|could|could'.*|couldn't|coz|create.*|creating|cuz|decid.*|defens.*|delay.*|deni.*|deny.*|depend|depended|depending|depends|describe|described|describes|describing|determina.*|determine|determined|determines|determining|digest.*|discern.*|discl.*|discover.*|disregard.*|done|doubt.*|duties|duty|effect.*|end|ended|ending.*|ends|enlighten.*|evaluat.*|examine.*|examining|expect.*|explain|explained|explaining|explains|explanat.*|explor.*|fact.*|feeling.*|feels|felt|figur.*|find.*|finish|fit|fits|forbid.*|forgiv.*|found|foundation.*|gather.*|generate.*|goal.*|grasp.*|guard.*|held|hence|hesitant|hesitat.*|hold|holding|holds|hope|hoped|hopef.*|hopes|hoping|how|how's|if|ignore.*|ignori.*|implic.*|incorporat.*|induc.*|infer|inferred|inferring|infers|influenc.*|inform|informs|inhib.*|initiat.*|insight.*|integrat.*|intell.*|interfer.*|justif.*|kind_of|kinda|knew|know|knowing|knowl.*|known|knows|learn.*|limit.*|meaning|meaningf.*|means|meant|mind.*|motivate.*|motive.*|must|need|needed|needing|needs|neglect.*|obstac.*|organize.*|organizing|origin|ought|outcome.*|perceiv.*|perception.*|ponder.*|pretty|prevent.*|produce.*|product|productive.*|prohib.*|purpose.*|question|questioning|questionned|questions|quit.*|rational.*|react.*|read|reading|reads|realiz.*|reason.*|reckon.*|recognis.*|recogniz.*|reconsider.*|reconstruct.*|reflect.*|refrain|refus.*|relate.*|relation.*|reluctan.*|remember.*|repress.*|require|required|requirement.*|requires|resolu.*|resolve|resolved|responsib.*|restrain.*|restrict.*|result.*|retard.*|rethink.*|reveal.*|rigid.*|root.*|saw|secret|secrets|see|seeing|seem|seemed|seems|settl.*|should|should'.*|shouldn't|since|smart.*|solution.*|solve.*|sort|sorta|source.*|stimul.*|stop|stopped|stopping|stops|structure.*|stubborn.*|suspect.*|therefor.*|think|thinking|thinks|thought|thoughts|thus|tried|tries|try|trying|understand|understandable|understanding|understands|understood|undo|unresolve.*|wait|waited|waiting|waits|want|wanted|wanting|wants|welcom.*|what|what's|why|why's|wish|wished|wishes|wishing|withheld|withhold|wonder|wondered|wondering|would|would'.*|wouldn'.*|yield.*")
motion= re.compile(r"^(action.*|advanc.*|approach|arrive|arrived|arrives|arriving|bring|bringing|brings|brought|carried|carries|carry|carrying|climb.*|cross|cruis.*|danc.*|deliver.*|depart|departs|disappear.*|drift.*|drive|driven|drives|driving|drove|enter|entered|entering|enters|explor.*|fled|flew|flies|fly|flying|follow|followed|following|follows|go|goes|going|gone|hik.*|jog.*|move|moved|moves|moving|pack|packed|packing|ran|run|running|runs|swim.*|take|takes|taking|took|transport.*|travel.*|visit.*|walk|walked|walking|walks|went)$")
tentative = re.compile(r"^(		alot|ambigu.*|any|anybod.*|anyhow|anyone.*|anything.*|anytime|anywhere|bet|bets|betting|conflict.*|confus.*|depend|depended|depending|depends|disorient.*|doubt.*|even|fortunat.*|fuzzy|guess|guessed|guesses|guessing|hazy|hesitant|hesitat.*|hope|hoped|hopef.*|hopes|hoping|just|kind_of|kinda|likel.*|luck|lucki.*|lucky|may|maybe|might|nearly|occasional.*|or|perhaps|possib.*|pretty|probab.*|puzzl.*|random|reckon.*|seem|seemed|seems|some|somebod.*|someone.*|something.*|sometime.*|sort|sorta|suppose|supposed|temporar.*|tentative.*|uncertain|unclear|undecided.*|unknow.*|unlikely|unresolve.*|unsure.*|vague.*|variable.*|wonder|wondered|wondering)$")
greeting = re.compile(r"\b(hey|hello|hi|howdy)\s+(guys|folks|all|everyone|everybody)\b")
greeting = re.compile(r"\b(hey|hello|hi|howdy)\b")


In [3]:

def handle_data(docs, dataset):

  #I think it is expecting a content and I am adding a label in place of scum, which i think is the deceptive or not indicator
  #pandas dataframe format??


    docs['words'] = docs['text'].apply(lambda c: re.findall(r"\w+", c.lower(), re.UNICODE))
    docs['wc'] = docs['words'].map(len)
    # this is expecting only ones over 50 words, so took it out bc many are shorter than that #docs = docs[docs['wc'] >= 50]
    cnt = lambda l, words: sum(l.count(w) for w in words)

    docs['punc'] = docs['text'].apply(lambda c: re.findall(r"[^\w\s]+", c.lower(), re.UNICODE))
    docs['or_ratio'] = docs['words'].apply(lambda d: d.count('or')) / docs['wc']
    docs['neg_em_ratio'] = docs['words'].apply(lambda d: sum(re.match(neg_emotion, word) != None for word in d)) / docs['wc']
    docs['anger_ratio'] = docs['words'].apply(lambda d: sum(re.match(anger, word) != None for word in d)) / docs['wc']
    docs['sensory_ratio'] = docs['words'].apply(lambda d: sum(re.match(sensory, word) != None for word in d)) / docs['wc']
    docs['cog_ratio'] = docs['words'].apply(lambda d: sum(re.match(cognitive, word) != None for word in d)) / docs['wc']
    docs['insight_ratio'] = docs['words'].apply(lambda d: sum(re.match(insight, word) != None for word in d)) / docs['wc']
    docs['motion_ratio'] = docs['words'].apply(lambda d: sum(re.match(motion, word) != None for word in d)) / docs['wc']
    docs['tent_ratio'] = docs['words'].apply(lambda d: sum(re.match(tentative, word) != None for word in d)) / docs['wc']
    print("regex done")

    docs['not_ratio'] = docs['words'].apply(lambda d: cnt_phrases(d, not_contractions) + cnt(d, other_negations)) / docs['wc']
    docs['fpp_ratio'] = docs['words'].apply(lambda d: cnt(d, ['i','we','my','mine','our','ours', 'myself', 'ourselves'])) / docs['wc']
    docs['spp_ratio'] = docs['words'].apply(lambda d: cnt(d, spp)) / docs['wc']
    docs['tpp_ratio'] = docs['words'].apply(lambda d: cnt(d, ['he','she','him','her','his','hers','they','them','their','theirs','themselves','himself','herself'])) / docs['wc']
    docs['quant_ratio'] = docs['words'].apply(lambda d: cnt(d, quantifiers)) / docs['wc']
    docs['sentence_length'] = docs['punc'].apply(lambda d: contains_any(d, ['.','!','?'])) / docs['wc']
    print("did things")

    docs['but_ratio'] = docs['words'].apply(lambda d: d.count('but')) / docs['wc']
    docs['token_length'] = docs['words'].apply(lambda d: sum(map(len, d))) / docs['wc']
    docs['unique_tokens'] = docs['words'].apply(lambda d: len(set(d))) / docs['wc']
    docs['message_count'] = docs['punc'].apply(lambda d: d.count('/!@') + 1)
    docs['message_length'] = docs['wc'] / docs['message_count']
    #this crashes docs['sentence_length'] = sum(docs['punc'].str.contains(r"\.|!|?")) / docs['wc']
    #docs['time'] = 1 + (docs['updated_at'] - docs['inserted_at']) / np.timedelta64(1, 'D')
    #docs['wc24h'] = docs['wc'] / docs['time']
    #docs['msg24h'] = docs['message_count'] / docs['time']
    print("finished w features")

    docs.to_csv("/content/drive/MyDrive/"+ dataset + ".tsv",sep='\t')

    #docs['vectorised'] = docs['words'].apply(lambda ws: np.mean([V(word) for word in ws if not V(word) is None], axis=0))



    return docs



In [5]:
import sklearn
from sklearn.model_selection import train_test_split
dataFile = "/content/drive/MyDrive/combopreppedMafia.tsv"
df=pd.read_csv(dataFile, sep='\t')
print(df)

df = df.drop(["game", "Unnamed: 0.1", "Unnamed: 0", "message", "withName", "history","mId", "role","name", "time", 'nohiscombo', 'msgAndHistory', 'Unnamed: 0.2' ], axis=1)
#df['msgAndHistory'] = df.apply(dataFix, axis = 1)
df = df.rename(columns={'label': 'labels','hiscombo': 'text'})
print(df)

#train, validate = train_test_split(df, test_size=0.1, random_state=42, shuffle=True)


#mtrain = Dataset.from_pandas(train)
#meval = Dataset.from_pandas(validate)

      Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0  mId             name       role  \
0                0             0         431    2   Natalie Morris      mafia   
1                1             1         422    3      Ryan Hodges      mafia   
2                2             2         432    4   Natalie Morris      mafia   
3                3             3         423    5      Ryan Hodges      mafia   
4                4             4         413   10    Mary Trujillo  bystander   
...            ...           ...         ...  ...              ...        ...   
2202          2202          2202          53   98    Nicole Barker  bystander   
2203          2203          2203          67  101    Angela Murphy  bystander   
2204          2204          2204          68  102    Angela Murphy  bystander   
2205          2205          2205          69  104  Richard Stevens      mafia   
2206          2206          2206          70  105  Jessica Michael      mafia   

                           

In [6]:
traindocsM = handle_data(df,"mafiawhistoryCombo" )

regex done
did things
finished w features


In [7]:
print(traindocsM)

     labels                                               text  \
0         D  {"Task" : "You will be given a message from th...   
1         D  {"Task" : "You will be given a message from th...   
2         D  {"Task" : "You will be given a message from th...   
3         D  {"Task" : "You will be given a message from th...   
4         T  {"Task" : "You will be given a message from th...   
...     ...                                                ...   
2202      T  {"Task" : "You will be given a message from th...   
2203      T  {"Task" : "You will be given a message from th...   
2204      T  {"Task" : "You will be given a message from th...   
2205      D  {"Task" : "You will be given a message from th...   
2206      D  {"Task" : "You will be given a message from th...   

                                                  words   wc  \
0     [task, you, will, be, given, a, message, from,...   83   
1     [task, you, will, be, given, a, message, from,...   86   
2     [task, yo

start here when files done

In [8]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [9]:
import sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#embeddings = model.encode(df['text'].tolist())


In [10]:
def handle_text(inFile, outFile):
  mafia = pd.read_csv(inFile, sep='\t')
  emb = model.encode(
    mafia['text'].tolist(),
    batch_size=64,  # Increase this based on your GPU memory
    show_progress_bar=True


)
#mafia['embeddings'] = list(emb)
  mafia['sentence_vector_sum'] = [np.sum(embedding) for embedding in emb]
  #mafia = mafia.drop(["words", "split",'text', 'punc' ], axis=1)

  mafia.to_csv(outFile,sep='\t')


In [12]:
handle_text('/content/drive/MyDrive/mafiawhistoryCombo.tsv', '/content/drive/MyDrive/mafiaWHistorywVecCombo.tsv')

Batches:   0%|          | 0/35 [00:00<?, ?it/s]

In [13]:
mafia = pd.read_csv('/content/drive/MyDrive/mafiaWHistorywVecCombo.tsv', sep='\t')
#mafia = mafia.drop(["Unnamed: 0.1", "Unnamed: 0"], axis=1)

In [14]:
print(mafia.dtypes)

Unnamed: 0.1             int64
Unnamed: 0               int64
labels                  object
text                    object
words                   object
wc                       int64
punc                    object
or_ratio               float64
neg_em_ratio           float64
anger_ratio            float64
sensory_ratio          float64
cog_ratio              float64
insight_ratio          float64
motion_ratio           float64
tent_ratio             float64
not_ratio              float64
fpp_ratio              float64
spp_ratio              float64
tpp_ratio              float64
quant_ratio            float64
sentence_length        float64
but_ratio              float64
token_length           float64
unique_tokens          float64
message_count            int64
message_length         float64
sentence_vector_sum    float64
dtype: object


In [15]:
mafia = mafia.drop(["Unnamed: 0.1", "Unnamed: 0", 'punc','text', 'words' ], axis=1)

In [16]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
import joblib

In [17]:
def trainSVM(df, dataset):

  y =  df["labels"]
  X = df.drop(["labels"], axis=1)

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)
  X_test_scaled = scaler.transform(X_test)

  svm_model = SVC(kernel='rbf', C=1.0)
  svm_model.fit(X_train_scaled, y_train)

  y_pred = svm_model.predict(X_test_scaled)
  print(accuracy_score(y_test, y_pred))
  print(classification_report(y_test, y_pred))

  joblib.dump(svm_model,'/content/drive/MyDrive/models/' + dataset + '.joblib')
  joblib.dump(scaler, '/content/drive/MyDrive/models/' + dataset +'Scaler.joblib')


In [18]:
#need to run through hander first before can do this
!pip install evaluate
import evaluate
import sklearn
import pandas as pd

acc = evaluate.load("accuracy")
confusion_metric = evaluate.load("confusion_matrix")
f1_metric = evaluate.load("f1")
recall = evaluate.load('recall')
precision = evaluate.load('precision')

def compute_metrics(predictions, labels):
    #predictions, labels = eval_pred
    fxlabels = {"T": 1, "D": 0}
    pred = [fxlabels[preds] for preds in predictions]
    print(pred)
    labels = [fxlabels[lbl] for lbl in labels.tolist()]
    df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/llm decetion/predictionsfixedwCombo.tsv", sep='\t')
    print(df)


    df['svmhistorypredcombo'] = pred
    df['svmhistorylabelsT1D0combo'] = labels
    df.to_csv("/content/drive/MyDrive/Colab Notebooks/llm decetion/predictionsfixedwCombo.tsv", sep='\t')
    print(labels)

   # pred = np.argmax(predictions, axis=-1)

    accuracy = acc.compute(predictions=pred, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=pred, references=labels)["f1"]
    cm = confusion_metric.compute(predictions=pred, references=labels)["confusion_matrix"]
    rc = recall.compute(predictions=pred, references=labels)["recall"]
    pr = precision.compute(predictions=pred, references=labels)["precision"]

    res = {
        "accuracy": accuracy,
        "f1": f1,
        "recall": rc,
        "recision": pr,
        "cm_true_positives": float(cm[0][0]),
        "cm_false_positives": float(cm[0][1]),
        "cm_false_negatives": float(cm[1][0]),
        "cm_true_negatives": float(cm[1][1]),
    }
    print(res)
    return res

def testSVm(model, scaler, dataset):
  model = joblib.load(model)
  scaler = joblib.load(scaler)

  y =  dataset["labels"]
  X = dataset.drop(["labels"], axis=1)
  Xsc = scaler.transform(X)
  yRes = model.predict(Xsc)

  compute_metrics(yRes, y)
  #print(accuracy_score(y, yRes))
  #print(classification_report(y, yRes))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00


In [19]:
trainSVM(mafia, "mafiawCombo" )

0.6561085972850679
              precision    recall  f1-score   support

           D       0.69      0.20      0.32       171
           T       0.65      0.94      0.77       271

    accuracy                           0.66       442
   macro avg       0.67      0.57      0.54       442
weighted avg       0.67      0.66      0.59       442



In [ ]:
#avalon set
av  = pd.read_csv("/content/drive/MyDrive/avalonWHistoryfixed.tsv", sep='\t')

print(av)
av = av.drop(["mid","persuasion", "deception","player","quest","turn","file", "Unnamed: 0.1", "Unnamed: 0", "msg", "withName", "history"], axis=1)
av = av.rename(columns={'label': 'labels','msgAndHistory': 'text'})
print(av.columns)
test = handle_data(av , "avalonsvmFixed")
print(test)


      Unnamed: 0.1  Unnamed: 0            mid         persuasion deception  \
0                0        1710  1VVGAqnFM9y0o          assertion      none   
1                1        1711    OJJPBkWizpG          assertion      none   
2                2        1712   nGGg4Vxtl2Xr          agreement      none   
3                3        1713    wnnK488C8mn          assertion      none   
4                4        1714  m77Y4VoFj974E          agreement      none   
...            ...         ...            ...                ...       ...   
2379          2379          36  xooqrjOHJqLzp  logical deduction      none   
2380          2380          37   1VV1vpEiqE9M  logical deduction      none   
2381          2381          38  nGGQ9mvu70A9O        questioning      none   
2382          2382          39  g77Y9KVhlpWLJ         suggestion      none   
2383          2383          40   JEED9jxUwokQ          assertion      none   

     label    player                                           

In [ ]:
ava = pd.read_csv("/content/drive/MyDrive/avalonsvmFixed.tsv", sep='\t')
emb = model.encode(
    ava['text'].tolist(),
    batch_size=64,  # Increase this based on your GPU memory
    show_progress_bar=True
)
#mafia['embeddings'] = list(emb)
ava['sentence_vector_sum'] = [np.sum(embedding) for embedding in emb]
ava = ava.drop(["words", 'text', 'punc' ], axis=1)

ava.to_csv("/content/drive/MyDrive/avalonsvmFixedVec.tsv",sep='\t')

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

In [ ]:
ava2 = ava.fillna(0)
ava[ava.isna().any(axis=1)]

,labels,wc,or_ratio,neg_em_ratio,anger_ratio,sensory_ratio,cog_ratio,insight_ratio,motion_ratio,tent_ratio,...,spp_ratio,tpp_ratio,quant_ratio,sentence_length,but_ratio,token_length,unique_tokens,message_count,message_length,sentence_vector_sum


In [ ]:
ava[ava.isna().any(axis=1)]
#ava.loc[712]#513 712

,labels,wc,or_ratio,neg_em_ratio,anger_ratio,sensory_ratio,cog_ratio,insight_ratio,motion_ratio,tent_ratio,...,spp_ratio,tpp_ratio,quant_ratio,sentence_length,but_ratio,token_length,unique_tokens,message_count,message_length,sentence_vector_sum


In [20]:
#handle_text("/content/drive/MyDrive/avalon.tsv", "/content/drive/MyDrive/avalonwVec.tsv")
ava = pd.read_csv("/content/drive/MyDrive/avalonsvmFixedVec.tsv", sep='\t')
#ava['tokenised_words'] = ava['words'].apply(sum_bert_tokens)
ava = ava.drop(["Unnamed: 0.1",'Unnamed: 0'], axis=1)
#ava = ava.replace([np.inf, -np.inf], np.nan)

#ava = ava.fillna(0)

#ava.fillna(ava.mean())
#ava = ava.dropna()
#print(ava.loc[712])
print(ava)

     labels   wc  or_ratio  neg_em_ratio  anger_ratio  sensory_ratio  \
0         T   20  0.000000      0.000000     0.000000       0.000000   
1         T   26  0.000000      0.000000     0.000000       0.038462   
2         T   30  0.000000      0.000000     0.000000       0.033333   
3         T   42  0.000000      0.000000     0.000000       0.023810   
4         T   53  0.000000      0.000000     0.000000       0.037736   
...     ...  ...       ...           ...          ...            ...   
2379      T  561  0.005348      0.028520     0.007130       0.001783   
2380      T  580  0.005172      0.032759     0.006897       0.001724   
2381      T  607  0.004942      0.032949     0.008237       0.001647   
2382      T  638  0.004702      0.032915     0.007837       0.001567   
2383      T  668  0.004491      0.031437     0.007485       0.001497   

      cog_ratio  insight_ratio  motion_ratio  tent_ratio  ...  spp_ratio  \
0      0.050000       0.000000      0.050000    0.050000  .

In [21]:
import joblib
testSVm('/content/drive/MyDrive/models/mafia.joblib', '/content/drive/MyDrive/models/mafiaScaler.joblib', ava)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

Ignore below

In [ ]:
unidecor = pd.read_csv("/content/drive/MyDrive/unidecor.tsv", sep='\t')
unidecor['tokenised_words'] = unidecor['words'].apply(sum_bert_tokens)
unidecor = unidecor.drop(["words","split",'text'], axis=1)


In [ ]:
def sum_bert_tokens(word_list):
    tokens = []
    for word in word_list:
        word_tokens = tokenizer.encode(word, add_special_tokens=False)
        tokens.append(sum(word_tokens))
    return sum(tokens)


In [ ]:
from matplotlib import pyplot as plt
from sklearn import svm
import pickle
from sklearn.metrics import accuracy_score

def f_importances(coef, names):
  #https://stackoverflow.com/questions/41592661/determining-the-most-contributing-features-for-svm-classifier-in-sklearn
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

def train_svm(train_docs, dataset):

  columns = list(train_docs.columns.values)
  print(columns)
  columns.remove('truth_label')


  #columns.remove('text')

  X = [[columns]]
  scale(X, copy=False)

  Y = train_docs['labels'].values

  features_names = docs.columns
  svm = svm.SVC(kernel='linear')
  svm.fit(X, Y)

  with open(dataset+ "_svm.pkl", 'wb') as f:
    pickle.dump(svm, f)



def test_svm(test_docs, dataset):
  X = test_docs[columns].values
  scale(X, copy=False)

  Y = test_docs['labels'].values

  with open(dataset+"_svm.pkl", "rb") as f:
    svm = pickle.load(f)

  pred = svm.predict(X)
  acc = accuracy_score(Y, pred)
  print(acc)


#f_importances(svm.coef_, features_names)


In [ ]:
#train_docs_mafia = handle_data(mafia)
#train_docs_uni = handle_data(unidecor)
print(train_docs_mafia)

train_svm(train_docs_mafia, "mafia")
#train_svm(train_docs_uni, "unidecor")

test = pd.read_json('/content/consolidated_deception_data.json')

test_docs = handle_data(test)
test_svm(test_docs, "mafia")
#test_svm(test_docs, "unidecor")

                                                    text truth_label split  \
0      excellent staff and customer service, very cle...           D  None   
1      My stay at this hotel was one of the best I ha...           D  None   
2      We just got back from a trip to Chicago for my...           D  None   
3      I have to say that the Hard Rock Hotel in Chic...           D  None   
4      My husband and I recently stayed at the Hard R...           D  None   
...                                                  ...         ...   ...   
50809  I'm going on a walk with a friend of mine and ...           D  None   
50810  I am going to a museum in Manchester that focu...           D  None   
50811  We used to be housemates and always watched th...           D  None   
50812  The RAF 100 Bicycle Trail is a commemorative t...           D  None   
50813  Going to meet up with friends. Have lunch and ...           D  None   

                                                   words   wc  

ValueError: could not convert string to float: 'text'

Ignore below

In [ ]:


docs = handle_data(df)
all_docs = docs
columns = "wc24h or_ratio tpp_ratio spp_ratio fpp_ratio but_ratio token_length unique_tokens message_length sentence_length msg24h anger_ratio sensory_ratio cog_ratio insight_ratio motion_ratio not_ratio quant_ratio neg_em_ratio tent_ratio".split()


#docs = docs.drop_duplicates(['slot_id'], keep=False)
X = docs[columns].values
scale(X, copy=False)

y = docs['labels'].values

for i, x in enumerate(X.T):
    print(columns[i])
    plt.figure()
    docs[columns[i]].hist()
    u, p = scipy.stats.mannwhitneyu(x[y], x[~y])
    r = roc_auc_score(y, x)
    r = str(round(r, 3))
    c = columns[i].replace('_', ' ')
    print(r'\textbf{' + c + '}' if p < 0.05 else c, "&", r'\textbf{' + r + '}' if p < 0.05 else r, r'\tabularnewline')

scoring = {s:s for s in ['f1', 'accuracy', 'precision', 'recall', 'roc_auc', 'average_precision']}
curves = []
curves_i = -1

def roc_curve_as_score(y, y_pred, **kwargs):
    global curves, curves_i
    curves_i += 1
    curves.append(roc_curve(y, y_pred))
    # print(curves[curves_i][2])
    return curves_i

def pr_curve(y, y_pred, **kwargs):
    global curves, curves_i
    curves_i += 1
    curves.append(precision_recall_curve(y, y_pred))
    return curves_i

scoring['roc_curve'] = make_scorer(roc_curve_as_score, needs_threshold=True)
scoring['pr_curve'] = make_scorer(pr_curve, needs_threshold=True)

## Evaluation

res = pd.DataFrame(columns=['name', 'wc', 'N', 'replacements', 'scores'])
wc = -1000

def evaluate(X, y, weights, res, name):
    cv = RepeatedStratifiedKFold(n_splits=20, n_repeats=1)
    for train_index, test_index in cv.split(X, y):
        # model = svm.LinearSVC(class_weight='balanced', max_iter=10000)
        model = linear_model.LogisticRegression(class_weight='balanced', solver='lbfgs', max_iter=1000)
        model.fit(X[train_index], y[train_index], weights[train_index])
        y_pred[test_index] = model.predict_proba(X[test_index])[:,1]

    scores = {'test_average_precision': average_precision_score(y, y_pred), 'test_roc_auc': roc_auc_score(y, y_pred), 'interval': delong.interval(y, y_pred)}
    res = res.append({'name': name, 'wc': 'all', 'N': N, 'replacements': replacements, 'scores': scores}, ignore_index=True)

    c = docs['wc'] > 5000

    scores = {'test_average_precision': average_precision_score(y[c], y_pred[c]), 'test_roc_auc': roc_auc_score(y[c], y_pred[c]), 'interval': delong.interval(y[c], y_pred[c])}
    res = res.append({'name': name, 'wc': '5000+', 'N': sum(c), 'replacements': replacements, 'scores': scores}, ignore_index=True)

    for wc_cohort in range(wc, 11000, 2000):
        cohort = (docs['wc'] >= wc_cohort) & (docs['wc'] < wc_cohort + 2000)
        c = cohort.values
        scores = {'test_average_precision': average_precision_score(y[c], y_pred[c]), 'test_roc_auc': roc_auc_score(y[c], y_pred[c]), 'interval': delong.interval(y[c], y_pred[c])}
        res = res.append({'name': name, 'wc': wc_cohort, 'N': sum(c), 'replacements': replacements, 'scores': scores}, ignore_index=True)
    c = docs['wc'] > 11000

    scores = {'test_average_precision': average_precision_score(y[c], y_pred[c]), 'test_roc_auc': roc_auc_score(y[c], y_pred[c]), 'interval': delong.interval(y[c], y_pred[c])}
    res = res.append({'name': name, 'wc': '11000+', 'N': sum(c), 'replacements': replacements, 'scores': scores}, ignore_index=True)
    return res


for replacements in [True]:
    # for wc in [5000]:
    docs = all_docs[all_docs['wc'] >= wc]
    print("CH")
    print(sum(docs['scum'])/len(docs))

    if not replacements:
        docs = docs.drop_duplicates(['slot_id'], keep=False)

    y = docs['scum'].values

    N = len(docs)

    X_hp = docs[columns].values
    scale(X_hp, copy=False)

    def my_scorer(y, y_pred, **kwargs):
        print(docs.iloc[np.argmax(y_pred)])
        return 0

    y_pred = np.zeros(len(y))

    # model = dummy.DummyClassifier()
    print('hey')
    # scores = cross_validate(model, X, y, cv=cv, scoring=scoring, return_train_score=False, return_estimator=True)

    res = evaluate(X_hp, y, np.log(docs['wc'].values), res, 'Hand-picked')


    for m, dataset in wv:
        X = np.stack(docs['vector_' + m + '_' + dataset].values)
        res = evaluate(X, y, np.log(docs['wc'].values), res, m)
        # model = MultinomialNB()
        # model = dummy.DummyClassifier()
        # avg precision = AU PRC

        X = np.concatenate((X_hp, np.stack(docs['vector_' + m + '_' + dataset].values)), axis=1)
        res = evaluate(X, y, np.log(docs['wc'].values), res, 'HP + ' + m)

res.to_pickle('res.pkl')
# if not USE_CACHED_DOCS:
# all_docs.to_pickle('docs.pkl', compression='gzip')

## res

results = res

results['AUROC'] = results['scores'].apply(lambda s: round(s['test_roc_auc'], 3))
results['AP'] = results['scores'].apply(lambda s: round(s['test_average_precision'], 3))
results['interval'] = results['scores'].apply(lambda s: np.round(s['interval'], 3))
results['wc'] = results['wc'].apply(lambda x: x if type(x) == np.str else str(max(x, 50)) + '-' + str(x + 1999))
range_res = results.loc[results['name'] == 'HP + FastText', ['wc', 'N', 'AUROC', 'AP']]

results['AUROC'] = results.apply(lambda x: '{AUROC} [{interval[0]}, {interval[1]}]'.format(**x), axis=1)
results = results.loc[(results['wc'] == 'all'), ['name', 'AUROC', 'AP']]

print()
print(range_res.to_latex(index=False))
print(results.to_latex(index=False))

print(results)
print(range_res)
